In [23]:
import os
import json
import glob
from typing import List

from debugpy.common.timestamp import reset

In [24]:
def process_json_files(directory, batch_size=5):
    """
    Reads JSON files, extracts data, and formats it for a LaTeX table.

    Args:
        directory (str): The path to the directory containing JSON files.
        batch_size (int): The number of JSON files to process per row.
    """
    # Use glob to find all JSON files in the specified directory
    json_files = sorted(glob.glob(os.path.join(directory, "*.json")))

    if not json_files:
        print(f"No JSON files found in '{directory}'")
        return

    latex_rows = []

    # Process files in batches
    for i in range(0, len(json_files), batch_size):
        batch = json_files[i:i + batch_size]
        latex_row_data = [f"{2 ** (4 - i // batch_size)}"]

        for file_path in batch:
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)

                # Extract the required fields
                # nl_residual from the first event's last iteration
                nl_residual_str = data["events"][0]["iterations"][-1]["nl_residual"]

                # total_iterations from the second event
                total_iterations = data["events"][1]["total_iterations"]

                # Check if nl_residual is a string (e.g., in scientific notation) and convert if necessary
                if isinstance(nl_residual_str, str):
                    nl_residual = float(nl_residual_str)
                else:
                    nl_residual = nl_residual_str

                # Format the nl_residual to one decimal place in scientific notation
                formatted_nl_residual = "{:.1e}".format(nl_residual)

                # Create the formatted string with the LaTeX command and parentheses
                formatted_entry = f"{formatted_nl_residual} \\iter{{{total_iterations}}}"
                latex_row_data.append(formatted_entry)

            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error processing file {file_path}: {e}")
                continue

        # Join the formatted entries with '&' for LaTeX table format
        latex_row = " & ".join(latex_row_data)

        latex_rows.append(latex_row)

    for row in latex_rows[::-1]:
        print(f"{row} \\\\")

directory_path = "sweeps"
process_json_files(directory_path)

1 & 4.0e-13 \iter{32} & 4.0e-13 \iter{32} & 4.0e-13 \iter{32} & 4.0e-13 \iter{32} & 4.0e-13 \iter{32} \\
2 & 4.9e+01 \iter{36} & 5.9e-01 \iter{32} & 1.5e-03 \iter{34} & 3.7e-06 \iter{32} & 1.7e-08 \iter{32} \\
4 & 8.9e+01 \iter{38} & 1.5e+00 \iter{32} & 5.0e-02 \iter{34} & 3.9e-03 \iter{32} & 2.3e-05 \iter{32} \\
8 & 1.3e+02 \iter{41} & 3.4e+00 \iter{34} & 1.9e-01 \iter{34} & 1.4e-02 \iter{31} & 3.9e-04 \iter{32} \\
16 & 1.9e+02 \iter{36} & 6.0e+00 \iter{30} & 3.7e-01 \iter{32} & 3.7e-02 \iter{32} & 2.3e-03 \iter{32} \\


In [64]:
matrix_list_path = "matrix_list.txt"
res_dir = "res"

def get_iterations_string(event: dict) -> str:
    """
    Extracts the total iterations from an event and appends a '+' if
    the stop reason is 'max_iterations_reached'.
    """
    iterations = event['total_iterations']
    stop_reason = event['stop_reason']
    if stop_reason == "max_iterations_reached":
        # return f"\\iter{{{iterations}\\textsuperscript{{+}}}}"
        return "\\iter{*}"
    return f"\\iter{{{iterations}}}"

def process_matrix_data(matrix_name: str, res_dir: str) -> List[str]:
    """
    Reads and processes three JSON files for a given matrix.

    Args:
        matrix_name (str): The name of the matrix.
        res_dir (str): The directory where the JSON files are located.

    Returns:
        List[str]: A list of formatted strings for a single table row.
    """
    files = sorted(glob.glob(os.path.join(res_dir, f"{matrix_name}_*.json")))

    if len(files) != 3:
        raise ValueError(f"Expected 3 JSON files for {matrix_name}, but found {len(files)}.")

    sdd_file, ddd_file, ddo_file = files

    sdd_data = json.load(open(sdd_file))
    ddd_data = json.load(open(ddd_file))
    ddo_data = json.load(open(ddo_file))

    # Get the relative_error for each config
    sdd_rel_error = sdd_data["relative_error"]
    ddd_rel_error = ddd_data["relative_error"]
    ddo_rel_error = ddo_data["relative_error"]

    # Get the relative_residual and formatted iterations string for each config
    sdd_rel_residual = sdd_data["relative_residual"]
    sdd_iterations_str = get_iterations_string(sdd_data["events"][-1])

    ddd_rel_residual = ddd_data["relative_residual"]
    ddd_iterations_str = get_iterations_string(ddd_data["events"][-1])

    ddo_rel_residual = ddo_data["relative_residual"]
    ddo_iterations_str = get_iterations_string(ddo_data["events"][-1])

    return [
        f"{{\\footnotesize \\texttt{{{matrix_name[:8].replace("_", "\\_")}}}}}",
        f"{sdd_rel_error:.1e}",
        f"{ddd_rel_error:.1e}",
        f"{ddo_rel_error:.1e}",
        f"{sdd_rel_residual:.1e} {sdd_iterations_str}",
        f"{ddd_rel_residual:.1e} {ddd_iterations_str}",
        f"{ddo_rel_residual:.1e} {ddo_iterations_str}"
    ]

def res():
    """
    Main function to read matrix names, process JSON data, and print a LaTeX table.
    """

    try:
        with open(matrix_list_path, "r") as f:
            matrix_names = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"Error: Matrix list file not found at '{matrix_list_path}'.")
        return

    for name in matrix_names:
        try:
            row_data = process_matrix_data(name, res_dir)
            print(" & ".join(row_data) + " \\\\")
        except (ValueError, FileNotFoundError, json.JSONDecodeError, KeyError) as e:
            print(f"Error processing {name}: {e}")
            continue

res()

{\footnotesize \texttt{1138\_bus}} & 9.3e-12 & 8.9e-12 & 6.6e-12 & 6.2e-15 \iter{187} & 3.7e-15 \iter{43} & 2.5e-15 \iter{48} \\
{\footnotesize \texttt{2cubes\_s}} & 1.8e-15 & 1.8e-15 & 1.3e-15 & 3.2e-16 \iter{2} & 3.2e-16 \iter{2} & 2.1e-16 \iter{2} \\
{\footnotesize \texttt{494\_bus}} & 2.6e-12 & 2.0e-12 & 1.5e-12 & 1.7e-15 \iter{47} & 1.7e-15 \iter{94} & 3.3e-15 \iter{16} \\
{\footnotesize \texttt{Dubcova3}} & 1.4e-13 & 1.4e-13 & 1.7e-13 & 7.9e-16 \iter{9} & 7.9e-16 \iter{9} & 5.3e-16 \iter{10} \\
{\footnotesize \texttt{G3\_circu}} & 1.0e-11 & 2.7e-11 & 4.8e-12 & 5.0e-16 \iter{63} & 5.0e-16 \iter{60} & 5.0e-16 \iter{64} \\
{\footnotesize \texttt{bcsstk08}} & 3.0e-12 & 1.1e-12 & 6.7e-12 & 4.0e-15 \iter{2} & 2.1e-15 \iter{19} & 6.3e-16 \iter{2} \\
{\footnotesize \texttt{bcsstk09}} & 1.3e+00 & 1.6e+00 & 1.6e+00 & 5.3e-01 \iter{*} & 6.1e-01 \iter{*} & 6.0e-01 \iter{*} \\
{\footnotesize \texttt{bcsstk14}} & 2.3e-08 & 3.1e-09 & 2.5e-09 & 1.2e-12 \iter{*} & 1.6e-13 \iter{*} & 1.3e-13 \iter

In [70]:
def get_ast_string(event: dict) -> str:
    """
    Extracts the total iterations from an event and appends a '+' if
    the stop reason is 'max_iterations_reached'.
    """
    stop_reason = event['stop_reason']
    if stop_reason == "max_iterations_reached":
        return " \\iter{*}"
    return ""

def get_timing_data(matrix_name: str, res_dir: str) -> List[str]:
    """
    Reads and processes three JSON files for a given matrix to extract timing data.

    Args:
        matrix_name (str): The name of the matrix.
        res_dir (str): The directory where the JSON files are located.

    Returns:
        List[str]: A list of formatted strings for a single timing table row.
    """
    files = sorted(glob.glob(os.path.join(res_dir, f"{matrix_name}_*.json")))

    if len(files) != 3:
        raise ValueError(f"Expected 3 JSON files for {matrix_name}, but found {len(files)}.")

    sdd_file, ddd_file, ddo_file = files

    sdd_data = json.load(open(sdd_file))
    ddd_data = json.load(open(ddd_file))
    ddo_data = json.load(open(ddo_file))

    sdd_fact_time_ms = sdd_data["events"][0]["total_time_ms"]
    ddd_fact_time_ms = ddd_data["events"][0]["total_time_ms"]
    ddo_fact_time_ms = ddo_data["events"][0]["total_time_ms"]

    sdd_solve_time_ms = sdd_data["events"][1]["total_time_ms"]
    sdd_iterations_str = get_ast_string(sdd_data["events"][-1])
    ddd_solve_time_ms = ddd_data["events"][1]["total_time_ms"]
    ddd_iterations_str = get_ast_string(ddd_data["events"][-1])
    ddo_solve_time_ms = ddo_data["events"][1]["total_time_ms"]
    ddo_iterations_str = get_ast_string(ddo_data["events"][-1])

    return [
        f"{{\\footnotesize \\texttt{{{matrix_name.replace("_", "\\_")}}}}}",
        f"{sdd_fact_time_ms / 1000.0:.2f}",
        f"{ddd_fact_time_ms / 1000.0:.2f}",
        f"{ddo_fact_time_ms / 1000.0:.2f}",
        f"{sdd_solve_time_ms / 1000.0:.2f}{sdd_iterations_str}",
        f"{ddd_solve_time_ms / 1000.0:.2f}{ddd_iterations_str}",
        f"{ddo_solve_time_ms / 1000.0:.2f}{ddo_iterations_str}",
    ]

def timing():
    try:
        with open(matrix_list_path, "r") as f:
            matrix_names = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"Error: Matrix list file not found at '{matrix_list_path}'.")
        return
    for name in matrix_names:
        try:
            row_data = get_timing_data(name, res_dir)
            print(" & ".join(row_data) + " \\\\")
        except (ValueError, FileNotFoundError, json.JSONDecodeError, KeyError) as e:
            print(f"Error processing {name}: {e}")
            continue
timing()

{\footnotesize \texttt{1138\_bus}} & 0.02 & 0.01 & 0.01 & 0.59 & 0.15 & 0.19 \\
{\footnotesize \texttt{2cubes\_sphere}} & 0.50 & 0.48 & 0.48 & 1.59 & 1.62 & 2.70 \\
{\footnotesize \texttt{494\_bus}} & 0.01 & 0.01 & 0.01 & 0.06 & 0.12 & 0.03 \\
{\footnotesize \texttt{Dubcova3}} & 10.94 & 11.96 & 11.87 & 15.95 & 17.41 & 27.67 \\
{\footnotesize \texttt{G3\_circuit}} & 0.42 & 0.47 & 0.46 & 588.19 & 566.95 & 711.52 \\
{\footnotesize \texttt{bcsstk08}} & 0.02 & 0.01 & 0.01 & 0.01 & 0.07 & 0.03 \\
{\footnotesize \texttt{bcsstk09}} & 0.03 & 0.01 & 0.03 & 0.69 \iter{*} & 1.04 \iter{*} & 1.91 \iter{*} \\
{\footnotesize \texttt{bcsstk14}} & 0.03 & 0.02 & 0.02 & 1.87 \iter{*} & 1.92 \iter{*} & 4.84 \iter{*} \\
{\footnotesize \texttt{bcsstk18}} & 0.05 & 0.09 & 0.05 & 1.43 & 1.69 & 2.65 \\
{\footnotesize \texttt{bloweybq}} & 0.03 & 0.04 & 0.03 & 5.45 \iter{*} & 0.09 & 0.23 \\
{\footnotesize \texttt{cbuckle}} & 0.22 & 0.24 & 0.23 & 5.21 & 1.08 & 2.03 \\
{\footnotesize \texttt{crystm01}} & 0.04 & 0.03